In [355]:
import pandas as pd
import numpy as np
from utils import *
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
from scipy.special import expit


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
train_data,test_data = get_data()

In [53]:
train_data

,match_id,season,venue,innings,ball,striker,non_striker,bowler,runs_off_bat,extras,...,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,wicket,wickets_left,total_runs,runs_left
50,598068,2013,M Chinnaswamy Stadium,2,0.1,MEK Hussey,M Vijay,R Rampaul,4,0,...,0.0,0.0,0,NaN,NaN,NaN,0,10,4,107.0
51,598068,2013,M Chinnaswamy Stadium,2,0.2,MEK Hussey,M Vijay,R Rampaul,1,0,...,0.0,0.0,0,NaN,NaN,NaN,0,10,1,103.0
52,598068,2013,M Chinnaswamy Stadium,2,0.3,M Vijay,MEK Hussey,R Rampaul,1,0,...,0.0,0.0,0,NaN,NaN,NaN,0,10,1,102.0
53,598068,2013,M Chinnaswamy Stadium,2,0.4,MEK Hussey,M Vijay,R Rampaul,0,0,...,0.0,0.0,0,NaN,NaN,NaN,0,10,0,101.0
54,598068,2013,M Chinnaswamy Stadium,2,0.5,MEK Hussey,M Vijay,R Rampaul,1,0,...,0.0,0.0,0,NaN,NaN,NaN,0,10,1,101.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,598065,2013,"Rajiv Gandhi International Stadium, Uppal",2,19.2,KK Cooper,PV Tambe,DW Steyn,4,0,...,0.0,0.0,0,NaN,NaN,NaN,0,2,4,33.0
241,598065,2013,"Rajiv Gandhi International Stadium, Uppal",2,19.3,KK Cooper,PV Tambe,DW Steyn,4,0,...,0.0,0.0,0,NaN,NaN,NaN,0,2,4,29.0
242,598065,2013,"Rajiv Gandhi International Stadium, Uppal",2,19.4,KK Cooper,PV Tambe,DW Steyn,0,0,...,0.0,0.0,caught,KK Cooper,NaN,NaN,1,2,0,25.0
243,598065,2013,"Rajiv Gandhi International Stadium, Uppal",2,19.5,PV Tambe,SK Trivedi,DW Steyn,1,0,...,0.0,0.0,0,NaN,NaN,NaN,0,1,1,25.0


In [9]:
    # df_cleaned = df_cleaned.loc[(df_cleaned.noballs == 0) & (df_cleaned.wides == 0)] #select only those that are not noballs or wides
((train_data.noballs == 0) & (train_data.wides == 0)).index.values


array([ 50,  51,  52, ..., 242, 243, 244])

In [32]:
DLR = pd.read_csv("DLR.csv")

In [33]:
DLR

,0,1,2,3,4,5,6,7,8,9
0,100.0,96.8,92.6,86.7,78.8,68.2,54.4,37.5,21.3,8.3
1,96.1,93.3,89.2,83.9,76.7,66.6,53.5,37.3,21.0,8.3
2,92.2,89.6,85.9,81.1,74.2,65.0,52.7,36.9,21.0,8.3
3,88.2,85.7,82.5,77.9,71.7,63.3,51.6,36.6,21.0,8.3
4,84.1,81.8,79.0,74.7,69.1,61.3,50.4,36.2,20.8,8.3
5,79.9,77.9,75.3,71.6,66.4,59.2,49.1,35.7,20.8,8.3
6,75.4,73.7,71.4,68.0,63.4,56.9,47.7,35.2,20.8,8.3
7,71.0,69.4,67.3,64.5,60.4,54.4,46.1,34.5,20.7,8.3
8,66.4,65.0,63.3,60.6,57.1,51.9,44.3,33.6,20.5,8.3
9,61.7,60.4,59.0,56.7,53.7,49.1,42.4,32.7,20.3,8.3


In [105]:
def get_RU(balls,wickets_left,table = DLR):

    RU = list(map(lambda b,w:DLR.iloc[int(b),10-w],balls,wickets_left))
    return np.array(RU)

In [106]:
get_RU([0.1,0.2,0.3,19.1],[10,9,8,10])

array([100. ,  96.8,  92.6,   6.4])

In [57]:
dummy_df = train_data.query('match_id == 598068')

In [82]:
wicket_value = [1.30,1.35,1.40,1.45,1.38,1.18,0.98,0.79,0.59,0.39,0.19]

In [63]:
np.ones(len(dummy_df))[((dummy_df.noballs == 0) & (dummy_df.wides == 0)).tolist()]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [261]:
def calculate_PI_3(match_df,wicket_value = wicket_value):
    # match_df = df[match_id == id]
    balls = np.ones(len(match_df))
    balls[((match_df.noballs > 0) | (match_df.wides >0 )).tolist()] = 0


    balls_consumed = np.cumsum(balls) - balls #runs consumed before this ball is bowled
    
    # 'extras', 'wides', 'noballs',
    #    'byes', 'legbyes', 'penalty'

    balls_left = 120 - balls_consumed
    # print(balls_left)
    runs_left = match_df.runs_left.tolist()
    # print(balls_left[0],match_df.runs_left.tolist()[0])
    CRRR = np.array(list(map(lambda r,b:r*6/b,runs_left,balls_left)))  # (CRRR/balls_consumed)*6
   
    IRRR = runs_left[0]/20
    state_ratio = CRRR/IRRR
    RU = get_RU(match_df.ball,match_df.wickets_left)
    
    batsmen =[]
    for st,n_st in zip(match_df.striker,match_df.non_striker):
        if st not in batsmen:
            batsmen.append(st)
        if n_st not in batsmen:
            batsmen.append(n_st)
    
    # print(match_df.striker.unique(),match_df.non_striker.unique())
    batsmen_weights = dict(zip(batsmen,wicket_value))  #  batsman_weights = {unique_batsman : vals from table}
    
    wicket_weight = np.zeros(len(match_df))
    # if wicket == 1, wicket_weight[idx] = batsman_weight[batsman]
    wicket_weight[(match_df.wicket==1).tolist()] = [batsmen_weights[batter] for batter in match_df.query("wicket == 1")['player_dismissed'].tolist()]
    wicket_weight = np.cumsum(wicket_weight)/11

    pi_3 = list(map(lambda x,y:x*1/2*y,state_ratio,np.exp(RU/100) + np.exp(wicket_weight)))
    # match_df['balls_left'] = balls_left
    return pi_3,balls_left
    # 
    #

In [262]:
calculate_PI_3(dummy_df)

([1.8591409142295225,
  1.8747639471222075,
  1.8906517771825653,
  1.837890307548493,
  1.853734189510118,
  1.8698536172449882,
  1.8335114654052853,
  1.8381763728947011,
  1.854588661938404,
  1.9362473238742985,
  1.916136084855087,
  1.9337153149913724,
  1.9024834604827574,
  1.9202636797396058,
  1.9383793748314888,
  1.9568401307822647,
  1.9632303924575332,
  1.9822908817046938,
  1.9492623235093858,
  1.9685619504748249,
  2.0721722036569257,
  2.050300136188785,
  2.058112568895782,
  2.066086082689521,
  1.989163671046555,
  1.9572048087416523,
  1.9780261364942229,
  1.9857864831500416,
  2.007371118836455,
  2.0294301421203715,
  1.9910916577979818,
  2.0134634741777346,
  2.0363437409297545,
  2.0597499908254986,
  2.189671907506775,
  2.1675484818137605,
  2.1478977799975913,
  2.173776066503586,
  2.2002855307292397,
  2.212193045862562,
  2.2398454589358443,
  2.2525551887333846,
  2.2206029938250746,
  2.3527485696889023,
  2.3323629507946237,
  2.3466989878444253,


In [263]:
train_PI = []
for id in train_data['match_id'].unique():
    match_df = train_data.query("match_id == @id")
    # print(id)
    pi,balls_l = (calculate_PI_3(match_df))
    if balls_l[-1] ==0:
        print(id)

In [241]:
pd.set_option('display.max_rows', 20)

In [242]:
dummy_df = train_data.query("match_id == 548315")

In [243]:
np.sum(dummy_df.noballs == 1)

0

In [244]:
np.sum(dummy_df.wides == 1)

6

In [245]:
a,b = calculate_PI_3(dummy_df)


/var/folders/jp/l319lw651tb48gz1j9rnkrjm0000gn/T/ipykernel_7686/565197433.py:19: RuntimeWarning: divide by zero encountered in double_scalars
  CRRR = np.array(list(map(lambda r,b:r*6/b,runs_left,balls_left)))  # (CRRR/balls_consumed)*6


In [246]:
dummy_df = dummy_df.copy()

In [247]:
dummy_df['B'] = b

In [248]:
dummy_df.query("extras == 1")[['ball','wides','legbyes','extras']]

,ball,wides,legbyes,extras
189,11.2,1.0,0.0,1
190,11.3,0.0,0.0,1
197,12.3,1.0,0.0,1
209,14.2,1.0,0.0,1
215,15.1,1.0,0.0,1
217,15.3,1.0,0.0,1
222,15.8,0.0,1.0,1
231,17.3,1.0,0.0,1


In [249]:
dummy_df.tail(15)

,match_id,season,venue,innings,ball,striker,non_striker,bowler,runs_off_bat,extras,...,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,wicket,wickets_left,total_runs,runs_left,B
234,548315,2012,M Chinnaswamy Stadium,2,17.6,R Vinay Kumar,HV Patel,Shakib Al Hasan,1,0,...,0.0,0,NaN,NaN,NaN,0,3,1,61.0,14.0
235,548315,2012,M Chinnaswamy Stadium,2,17.7,HV Patel,R Vinay Kumar,Shakib Al Hasan,0,0,...,0.0,bowled,HV Patel,NaN,NaN,1,3,0,60.0,13.0
236,548315,2012,M Chinnaswamy Stadium,2,18.1,R Vinay Kumar,Z Khan,RN ten Doeschate,0,2,...,0.0,0,NaN,NaN,NaN,0,2,2,60.0,12.0
237,548315,2012,M Chinnaswamy Stadium,2,18.2,Z Khan,R Vinay Kumar,RN ten Doeschate,1,0,...,0.0,0,NaN,NaN,NaN,0,2,1,58.0,11.0
238,548315,2012,M Chinnaswamy Stadium,2,18.3,R Vinay Kumar,Z Khan,RN ten Doeschate,1,0,...,0.0,0,NaN,NaN,NaN,0,2,1,57.0,10.0
239,548315,2012,M Chinnaswamy Stadium,2,18.4,Z Khan,R Vinay Kumar,RN ten Doeschate,4,0,...,0.0,0,NaN,NaN,NaN,0,2,4,56.0,9.0
240,548315,2012,M Chinnaswamy Stadium,2,18.5,Z Khan,R Vinay Kumar,RN ten Doeschate,1,0,...,0.0,0,NaN,NaN,NaN,0,2,1,52.0,8.0
241,548315,2012,M Chinnaswamy Stadium,2,18.6,R Vinay Kumar,Z Khan,RN ten Doeschate,1,0,...,0.0,0,NaN,NaN,NaN,0,2,1,51.0,7.0
242,548315,2012,M Chinnaswamy Stadium,2,18.7,Z Khan,R Vinay Kumar,RN ten Doeschate,2,0,...,0.0,0,NaN,NaN,NaN,0,2,2,50.0,6.0
243,548315,2012,M Chinnaswamy Stadium,2,19.1,R Vinay Kumar,Z Khan,Shakib Al Hasan,1,0,...,0.0,0,NaN,NaN,NaN,0,2,1,48.0,5.0


In [250]:
dummy_df.columns

Index(['match_id', 'season', 'venue', 'innings', 'ball', 'striker',
       'non_striker', 'bowler', 'runs_off_bat', 'extras', 'wides', 'noballs',
       'byes', 'legbyes', 'penalty', 'wicket_type', 'player_dismissed',
       'other_wicket_type', 'other_player_dismissed', 'wicket', 'wickets_left',
       'total_runs', 'runs_left', 'B'],
      dtype='object')

In [251]:
dummy_df[((dummy_df.extras >0) & (dummy_df.wides == 0) & 
           (dummy_df.noballs == 0) & (dummy_df.byes == 0) & 
           (dummy_df.legbyes == 0) & (dummy_df.penalty == 0))]

,match_id,season,venue,innings,ball,striker,non_striker,bowler,runs_off_bat,extras,...,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,wicket,wickets_left,total_runs,runs_left,B


In [257]:
dummy_df.query("ball == 18.1")[['extras', 'wides', 'noballs', 'byes', 'legbyes', 'penalty']]

,extras,wides,noballs,byes,legbyes,penalty
236,2,2.0,0.0,0.0,0.0,0.0


# Model

In [266]:
train_PI = []
for id in train_data['match_id'].unique():
    match_df = train_data.query("match_id == @id")
    # print(id)
    pi,balls_l = (calculate_PI_3(match_df))
    train_PI.extend(pi)

In [285]:
test_PI = []
for id in test_data['match_id'].unique():
    match_df = test_data.query("match_id == @id")
    pi,balls_l = calculate_PI_3(match_df)
    test_PI.extend(pi)

In [270]:
class pi_3_model(torch.nn.Module):

    def __init__(self,input_size) -> None:
        super().__init__()
        self.comb_layer = torch.nn.Linear(input_size,1)
    
    def forward(self,x):

        out = self.comb_layer(x)
        reg_out = F.relu(out)

        return reg_out

In [273]:
encode_venue(train_data,test_data)

In [275]:
train_data.head()

,match_id,season,venue,innings,ball,striker,non_striker,bowler,runs_off_bat,extras,...,venue_36,venue_37,venue_38,venue_39,venue_40,venue_41,venue_42,venue_43,venue_44,venue_45
50,598068,2013,M Chinnaswamy Stadium,2,0.1,MEK Hussey,M Vijay,R Rampaul,4,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
51,598068,2013,M Chinnaswamy Stadium,2,0.2,MEK Hussey,M Vijay,R Rampaul,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52,598068,2013,M Chinnaswamy Stadium,2,0.3,M Vijay,MEK Hussey,R Rampaul,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53,598068,2013,M Chinnaswamy Stadium,2,0.4,MEK Hussey,M Vijay,R Rampaul,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54,598068,2013,M Chinnaswamy Stadium,2,0.5,MEK Hussey,M Vijay,R Rampaul,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [286]:
train_data['PI_3'] = train_PI
test_data['PI_3'] = test_PI

In [287]:
train_cols = ['innings', 'ball','wickets_left','runs_left','PI_3']
train_cols.extend([f'venue_{no}' for no in range(train_data.venue.nunique())])
target_cols = ['runs_off_bat']

In [288]:
train_data[train_cols]

,innings,ball,wickets_left,runs_left,PI_3,venue_0,venue_1,venue_2,venue_3,venue_4,...,venue_36,venue_37,venue_38,venue_39,venue_40,venue_41,venue_42,venue_43,venue_44,venue_45
50,2,0.1,10,107.0,1.859141,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
51,2,0.2,10,103.0,1.804679,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52,2,0.3,10,102.0,1.802304,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53,2,0.4,10,101.0,1.799887,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54,2,0.5,10,101.0,1.815403,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,2,19.2,2,33.0,10.124467,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
241,2,19.3,2,29.0,11.121574,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
242,2,19.4,2,25.0,13.274902,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
243,2,19.5,1,25.0,19.837498,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [336]:
train_dataset = torch.utils.data.TensorDataset(torch.Tensor(train_data[train_cols].values),torch.Tensor(train_data[target_cols].values))
test_dataset = torch.utils.data.TensorDataset(torch.Tensor(test_data[train_cols].values),torch.Tensor(test_data[target_cols].values))

In [344]:
train_loader = DataLoader(train_dataset,batch_size=len(train_dataset),shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=len(test_dataset))

In [338]:
device = torch.device('cpu')

In [339]:
pi_model = pi_3_model(len(train_cols))
pi_model.to(device)

pi_3_model(
  (comb_layer): Linear(in_features=51, out_features=1, bias=True)
)

In [340]:
loss_reg = torch.nn.MSELoss()
# loss_class = torch.nn.BCELoss()
optimizer = torch.optim.Adam(pi_model.parameters())

In [341]:
epochs = 500

In [342]:
for epoch in range(epochs):
    epoch_loss = 0
    # torch.nn.init.xavier_uniform_(model.pressure_layer.all_weights,gain= 'sigmoid')
    for batch,data in enumerate(train_loader):
        
        optimizer.zero_grad()
        x,target = data
        x = x.to(device)
        target = target.to(device)
 
        out = pi_model(x)

        loss = loss_reg(out,target)
 
        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()

    print(epoch_loss)


164.84947204589844
161.74581909179688
158.67404174804688
155.63470458984375
152.62832641601562
149.65541076660156
146.71644592285156
143.8118133544922
140.94198608398438
138.1073760986328
135.3083038330078
132.54515075683594
129.8181915283203
127.12771606445312
124.47405242919922
121.85737609863281
119.27791595458984
116.73587036132812
114.23139953613281
111.76461791992188
109.3355941772461
106.94441986083984
104.59111022949219
102.27564239501953
99.99803161621094
97.75817108154297
95.5560302734375
93.3915023803711
91.26445770263672
89.17478942871094
87.12230682373047
85.10682678222656
83.12818145751953
81.18614196777344
79.2804946899414
77.41092681884766
75.57717895507812
73.77892303466797
72.01585388183594
70.28768157958984
68.59407043457031
66.93465423583984
65.3091049194336
63.71706771850586
62.158164978027344
60.6319694519043
59.13810348510742
57.676212310791016
56.2458610534668
54.84661865234375
53.47805404663086
52.13975524902344
50.831321716308594
49.55228042602539
48.302272796

In [348]:
pi_model.eval()
for data in train_loader:
    x,target = data
    x = x.to(device)
    target = target.to(device)

    out = pi_model(x)
    print(mean_squared_error(train_data[target_cols],out.detach().numpy()))

3.722692501322215


In [346]:
pi_model.eval()
for data in test_loader:
    x,target = data
    x = x.to(device)
    target = target.to(device)

    out = pi_model(x)
    print(mean_squared_error(test_data[target_cols],out.detach().numpy()))
    

4.25984333409529


## PI-3 with sigmoid

In [369]:
train_data['PI_3_squashed'] = expit(train_data['PI_3'])
test_data['PI_3_squashed'] = expit(test_data['PI_3'])

In [376]:
model_2_train_cols = ['innings', 'ball','wickets_left','runs_left','PI_3_squashed']
model_2_train_cols.extend([f'venue_{no}' for no in range(train_data.venue.nunique())])


In [377]:
pi_model_2 = pi_3_model(len(model_2_train_cols))
pi_model_2.to(device)

pi_3_model(
  (comb_layer): Linear(in_features=51, out_features=1, bias=True)
)

In [378]:
train_dataset_2 = torch.utils.data.TensorDataset(torch.Tensor(train_data[model_2_train_cols].values),torch.Tensor(train_data[target_cols].values))
test_dataset_2 = torch.utils.data.TensorDataset(torch.Tensor(test_data[model_2_train_cols].values),torch.Tensor(test_data[target_cols].values))

In [379]:
train_loader_2 = DataLoader(train_dataset,batch_size=len(train_dataset),shuffle=True)
test_loader_2 = DataLoader(test_dataset,batch_size=len(test_dataset))

In [380]:
# loss_reg = torch.nn.MSELoss()
optimizer_2 = torch.optim.Adam(pi_model_2.parameters())

In [381]:
for epoch in range(epochs):
    epoch_loss = 0
    # torch.nn.init.xavier_uniform_(model.pressure_layer.all_weights,gain= 'sigmoid')
    for batch,data in enumerate(train_loader_2):
        
        optimizer_2.zero_grad()
        x,target = data
        x = x.to(device)
        target = target.to(device)
 
        out = pi_model_2(x)

        loss = loss_reg(out,target)
 
        loss.backward()

        optimizer_2.step()

        epoch_loss += loss.item()

    print(epoch_loss)


206.65150451660156
203.09901428222656
199.5786590576172
196.09092712402344
192.6363983154297
189.21556091308594
185.82891845703125
182.47689819335938
179.16000366210938
175.8786163330078
172.63314819335938
169.4239501953125
166.2514190673828
163.11587524414062
160.017578125
156.9568634033203
153.93392944335938
150.94900512695312
148.00230407714844
145.09397888183594
142.22418212890625
139.3930206298828
136.60057067871094
133.84693908691406
131.1321258544922
128.4561309814453
125.81896209716797
123.22058868408203
120.66096496582031
118.14000701904297
115.65762329101562
113.21367645263672
110.80805969238281
108.4405746459961
106.11109161376953
103.81941223144531
101.56529998779297
99.34857177734375
97.16898345947266
95.02627563476562
92.92017364501953
90.85042572021484
88.81675720214844
86.81886291503906
84.8564453125
82.92918395996094
81.03677368164062
79.17887115478516
77.35511779785156
75.56521606445312
73.80880737304688
72.08551788330078
70.39498138427734
68.73687744140625
67.1108093

In [382]:
pi_model_2.eval()
for data in test_loader:
    x,target = data
    x = x.to(device)
    target = target.to(device)

    out = pi_model_2(x)
    print(mean_squared_error(test_data[target_cols],out.detach().numpy()))
    

3.022182132312145
